## EOF Power spectrum with Julia

## import package

In [1]:
using DSP
using Plots
using NCDatasets
using LazyGrids
using Statistics
using FFTW
using LinearAlgebra
using DataStructures

## Load data

In [2]:
path::String = "/work/b11209013/2024_Research/MPAS/PC/CNTL_PC.nc"

dims = Dict{String, Array}()
data = Dict{String, Array}()

NCDataset(path, "r") do ds
    for key in keys(ds.dim)
        dims[key] = ds[key]
    end

    for key in keys(ds)
        if key in keys(dims)
            continue
        else
            data[key] = permutedims(ds[key][:, :, :, 1] + ds[key][:, :, :, 2], (3, 2, 1))
        end
    end
end

println(keys(data))

["q1", "rqvcuten", "sw", "cu", "t", "lw", "qv"]


## Processing data

### Symmetrize

In [3]:
function sym(
        lat::Array{Float64, 1},
        arr::Array{Float64, 3}
)
    latr:: Array{Float64, 1} = cos.(deg2rad.(lat))[1, :, 1];
    lats:: Float64           = sum(latr)

    sym_arr :: Array{Float64, 2} = dropdims(sum(arr .* latr, dims=2) / lats, dims=2)
    
    return sym_arr
end

function asy(
        lat::Array{Float64, 1},
        arr::Array{Float64, 3}
)
    latr:: Array{Float64, 1} = cos.(deg2rad.(lat))[1, :, 1];
    lats:: Float64           = sum(latr)

    data_asy::Array{Float64, 3} = arr .* latr
    data_asy[:, lat .< 0, :] *= -1

    data_asy_dropdims::Array{Float64, 2} = dropdims(sum(data_asy, dims=2)/lats, dims=2)
    return data_asy_dropdims
end

q1_sym::Array{Float64, 2}=sym(dims["lat"], data["q1"])
q1_asy::Array{Float64, 2}=asy(dims["lat"], data["q1"])

720×376 Matrix{Float64}:
   87.3147  -258.391     52.6443  …  -512.009   -258.105    319.557
   18.7925  -289.056    -27.7256     -573.987   -209.103    298.17
  139.355   -276.831   -225.256      -708.154   -292.504     97.2858
  179.19    -236.833   -476.881      -858.546   -314.19    -141.049
  229.177   -284.068   -660.139      -657.519   -205.003   -117.454
  327.522   -352.688   -683.048   …  -487.526   -148.0      -18.7775
  250.399   -354.213   -608.835      -472.557   -309.093    -45.1536
  -18.8471  -317.499   -603.229      -463.448   -579.552   -122.143
 -188.329   -276.162   -629.418      -411.891   -576.3     -257.666
 -228.857   -244.377   -456.41       -282.039   -485.754   -604.828
 -191.814   -330.052   -203.009   …   -67.782   -584.809   -838.118
 -130.683   -356.586   -117.078       162.309   -744.032   -727.378
 -216.112   -341.701   -105.218       413.237   -764.155   -535.897
    ⋮                             ⋱                           ⋮
  276.186   -327.378   -2

### Windowing

In [4]:
window_size::Float64 = 120.

taper::Array{Float64, 1} = hanning(120)

sym_window = stack(map(i -> q1_sym[:, (i-1)*60+1:(i-1)*60+120] .* taper[1, :], 1:5));
asy_window = stack(map(i -> q1_asy[:, (i-1)*60+1:(i-1)*60+120] .* taper[1, :], 1:5));

## Power Spectrum

In [15]:
wn::Array{Float64, 1} = fftfreq(size(sym_window, 1))
fr::Array{Float64, 1} = fftfreq(Int(window_size), 4)
println(size(fr))
frm, wnm = ndgrid(fr, wn)

function power_spec(
        data::Array{Float64, 3}
)
    data_fft::Array{ComplexF64, 3} = mapslices(fft, data; dims=1)
    data_fft = mapslices(ifft, data; dims=2)

    ps::Array{Float64 ,2} = (dropdims(mean(data_fft .* conj(data_fft), dims=3), dims=3))

    ps /= prod(size(ps))

    return log.(fftshift(ps))
end



(120,)


power_spec (generic function with 1 method)

In [16]:
sym_ps = power_spec(sym_window)
contourf(fftshift(wn), fftshift(fr), sym_ps')